# BÁO CÁO THỰC NGHIỆM: MÔ HÌNH MẠNG LƯỚI BAYESIAN
## Ứng dụng: Dự đoán Hiệu suất Học tập của Sinh viên

**Mục tiêu:**
1. Xây dựng mô hình Mạng lưới Bayesian (BN) với 5 nút.
2. Cài đặt và minh họa thuật toán suy luận chính xác (Enumeration-Ask) và xấp xỉ (Rejection Sampling).
3. Đánh giá ưu điểm và hạn chế của các phương pháp suy luận.

---

In [64]:
import random
import time
import pandas as pd
from IPython.display import display, Markdown, HTML
from dag_library import DAG

# --- Lớp Bayesian Network ---
class BayesianNetwork:
    def __init__(self, dag_instance):
        self.dag = dag_instance
        self.cpts = {} 
        self.nodes_vars = {} 

    def add_node_vars(self, node, possible_values):
        self.dag.add_node(node)
        self.nodes_vars[node] = possible_values

    def add_edge(self, u, v):
        self.dag.add_edge(u, v)

    def add_cpt(self, node, cpt_table):
        self.cpts[node] = cpt_table
    
    def get_parents(self, node):
        return [p for p, children in self.dag.adj.items() if node in children]

    # Thuật toán Suy luận Chính xác
    def enumeration_ask(self, query_var, evidence):
        def enumerate_all(variables, evidence_vars):
            if not variables: return 1.0
            first_var = variables[0]
            rest_vars = variables[1:]
            
            node_parents = self.get_parents(first_var)
            parent_states = tuple([evidence_vars.get(p) for p in node_parents])

            if first_var in evidence_vars:
                val = evidence_vars[first_var]
                prob = self.cpts[first_var][parent_states][val]
                return prob * enumerate_all(rest_vars, evidence_vars)
            else:
                total_prob = 0
                for val in self.nodes_vars[first_var]:
                    prob = self.cpts[first_var][parent_states][val]
                    new_evidence = evidence_vars.copy()
                    new_evidence[first_var] = val
                    total_prob += prob * enumerate_all(rest_vars, new_evidence)
                return total_prob

        Q = {val: 0.0 for val in self.nodes_vars[query_var]}
        ordered_vars = self.dag.topological_sort()

        for q_val in self.nodes_vars[query_var]:
            extended_evidence = evidence.copy()
            extended_evidence[query_var] = q_val
            Q[q_val] = enumerate_all(ordered_vars, extended_evidence)
        
        total = sum(Q.values())
        return {val: prob / total for val, prob in Q.items()}

    # Thuật toán Suy luận Xấp xỉ
    def rejection_sampling(self, query_var, evidence, num_samples):
        counts = {val: 0 for val in self.nodes_vars[query_var]}
        total_accepted_samples = 0
        
        for _ in range(num_samples):
            sample = {}
            is_consistent = True
            for node in self.dag.topological_sort():
                node_parents = self.get_parents(node)
                parent_states = tuple([sample.get(p) for p in node_parents])
                
                cpt = self.cpts[node][parent_states]
                values = list(cpt.keys())
                probabilities = list(cpt.values())
                
                sampled_val = random.choices(values, weights=probabilities, k=1)[0]
                sample[node] = sampled_val
                
                if node in evidence and sampled_val != evidence[node]:
                    is_consistent = False
                    break
            
            if is_consistent:
                counts[sample[query_var]] += 1
                total_accepted_samples += 1
        
        if total_accepted_samples == 0:
            return {val: 0.0 for val in counts}, total_accepted_samples
        
        return {val: count / total_accepted_samples for val, count in counts.items()}, total_accepted_samples

print("Các lớp DAG và BayesianNetwork đã được định nghĩa.")

Các lớp DAG và BayesianNetwork đã được định nghĩa.


## XÂY DỰNG MÔ HÌNH

**Bài toán:** Mạng lưới Dự đoán Hiệu suất Học tập (Student Performance)

| Biến | Ý nghĩa | Giá trị | Phụ thuộc
| :--- | :--- | :--- | :--- |
| **D** (Difficulty) | Độ khó | Easy, Hard |
| **I** (Intelligence) | Trí thông minh | Low, High |
| **G** (Grade) | Điểm thi | A, B, C | **D**, **I** |
| **S** (SAT) | Điểm SAT | Low, High | **I** |
| **L** (Letter) | Đánh giá cuối cùng | Weak, Strong | **G** |

**Cấu trúc:** $D \rightarrow G$, $I \rightarrow G$, $I \rightarrow S$, $G \rightarrow L$ (Tổng cộng 5 nút, 5 CPTs).

In [65]:
# --- Xây dựng mô hình Student Performance Network ---
dag = DAG()
bn = BayesianNetwork(dag)

# 1. Thêm các nút và giá trị
bn.add_node_vars("Intelligence", ["Low", "High"])
bn.add_node_vars("Difficulty", ["Easy", "Hard"])
bn.add_node_vars("Grade", ["A", "B", "C"])
bn.add_node_vars("SAT", ["Low", "High"])
bn.add_node_vars("Letter", ["Weak", "Strong"])

# 2. Thêm các cạnh
bn.add_edge("Intelligence", "Grade")
bn.add_edge("Difficulty", "Grade")
bn.add_edge("Intelligence", "SAT")
bn.add_edge("Grade", "Letter")

# 3. Thêm các CPT
bn.add_cpt("Difficulty", {(): {"Easy": 0.6, "Hard": 0.4}})
bn.add_cpt("Intelligence", {(): {"Low":0.7, "High":0.3}})

bn.add_cpt("Grade", {
    ("Low", "Easy"): {"A": 0.3, "B": 0.4, "C": 0.3},
    ("Low", "Hard"): {"A": 0.05, "B": 0.25, "C": 0.7},
    ("High", "Easy"): {"A": 0.9, "B": 0.08, "C": 0.02},
    ("High", "Hard"): {"A": 0.5, "B": 0.3, "C": 0.2}
})

bn.add_cpt("SAT", {
    ("Low",): {"Low": 0.95, "High": 0.05},
    ("High",): {"Low": 0.2, "High": 0.8}
})

bn.add_cpt("Letter", {
    ("A",): {"Weak": 0.1, "Strong": 0.9},
    ("B",): {"Weak": 0.4, "Strong": 0.6},
    ("C",): {"Weak": 0.99, "Strong": 0.01}
})

### 2.2. Trực quan hóa Bảng Xác suất Điều kiện (CPTs)

Sử dụng **Pandas DataFrame** để hiển thị các CPT của các nút có nút cha.

In [66]:
def visualize_cpt(bn_model, node_name):
    """Trực quan hóa CPT của một nút bằng DataFrame, không dùng canvas/vẽ."""
    node_cpt = bn_model.cpts.get(node_name)
    parents = bn_model.get_parents(node_name)
    
    if not node_cpt:
        print(f"Không tìm thấy CPT cho nút: {node_name}")
        return

    display(Markdown(f"#### CPT cho nút: **{node_name}**"))
    
    if not parents:
        df_data = {
            f"P({node_name})": list(node_cpt[()].keys()),
            "Xác suất": list(node_cpt[()].values())
        }
        df = pd.DataFrame(df_data)
        display(df.style.format({'Xác suất': '{:.2f}'}).hide(axis="index"))
    else:
        rows = []
        for parent_state_tuple, probabilities in node_cpt.items():
            row = dict(zip(parents, parent_state_tuple))
            row.update({val: prob for val, prob in probabilities.items()})
            rows.append(row)
        
        df = pd.DataFrame(rows)
        cols = parents + list(bn_model.nodes_vars[node_name])
        df = df[cols]
        
        format_dict = {col: '{:.2f}' for col in bn_model.nodes_vars[node_name]}
        display(df.style.hide(axis="index").format(format_dict).set_caption(f"P({node_name} | {', '.join(parents)})"))

visualize_cpt(bn, "Difficulty")
visualize_cpt(bn, "Intelligence")
visualize_cpt(bn, "SAT")
visualize_cpt(bn, "Grade")
visualize_cpt(bn, "Letter")

#### CPT cho nút: **Difficulty**

P(Difficulty),Xác suất
Easy,0.60
Hard,0.40


#### CPT cho nút: **Intelligence**

P(Intelligence),Xác suất
Low,0.70
High,0.30


#### CPT cho nút: **SAT**

Intelligence,Low,High
Low,0.95,0.05
High,0.20,0.80


#### CPT cho nút: **Grade**

Intelligence,Difficulty,A,B,C
Low,Easy,0.30,0.40,0.30
Low,Hard,0.05,0.25,0.70
High,Easy,0.90,0.08,0.02
High,Hard,0.50,0.30,0.20


#### CPT cho nút: **Letter**

Grade,Weak,Strong
A,0.10,0.90
B,0.40,0.60
C,0.99,0.01


## ỨNG DỤNG VÀ ĐÁNH GIÁ

In [ ]:
import time

# --- Kịch bản Suy luận ---
evidence = {"SAT": "High", "Difficulty": "Easy", "Intelligence": "Low"}
query = "Letter"
display(Markdown(f"### Kịch bản: Tính **P({query} | {evidence})**"))

# ----------------------------------
# Suy luận Chính xác (Enumeration-Ask)
# ----------------------------------
display(Markdown("#### Suy luận Chính xác (Enumeration-Ask)"))
start_time_exact = time.time()
exact_result = bn.enumeration_ask(query, evidence)
end_time_exact = time.time()

data_exact = {
    "Điểm": list(exact_result.keys()),
    "Xác suất": list(exact_result.values())
}
df_exact = pd.DataFrame(data_exact)
display(df_exact.style.format({'Xác suất': '{:.6f}'}).set_caption("Kết quả Chính xác").hide(axis="index"))
time_exact = end_time_exact - start_time_exact
print(f"Thời gian tính toán Chính xác: {time_exact:.6f} giây")


# ----------------------------------
# Suy luận Xấp xỉ (Rejection Sampling)
# ----------------------------------
display(Markdown("#### Suy luận Xấp xỉ (Rejection Sampling)"))
num_samples = 1000000 

start_time_approx = time.time()
approx_result, accepted_samples = bn.rejection_sampling(query, evidence, num_samples)
end_time_approx = time.time()
time_approx = end_time_approx - start_time_approx

display(Markdown(f"**Số mẫu tạo ra:** {num_samples} | **Số mẫu được chấp nhận:** {accepted_samples} ({(accepted_samples/num_samples)*100:.2f}%) "))

data_approx = {
    "Điểm": list(approx_result.keys()),
    "Xác suất": list(approx_result.values())
}
df_approx = pd.DataFrame(data_approx)
display(df_approx.style.format({'Xác suất': '{:.6f}'}).set_caption(f"Kết quả Xấp xỉ (N={num_samples})").hide(axis="index"))
print(f"Thời gian tính toán Xấp xỉ: {time_approx:.6f} giây")

display(Markdown("### So sánh"))

df_compare = pd.DataFrame({
    'Điểm': approx_result.keys(),
    'P(Chính xác)': [exact_result[k] for k in approx_result.keys()],
    'P(Xấp xỉ)': [approx_result[k] for k in approx_result.keys()],
    'Sai số |P_exact - P_approx|': [abs(exact_result[k] - approx_result[k]) for k in approx_result.keys()]
})

df_compare = df_compare.style.hide(axis="index").format({'P(Chính xác)': '{:.6f}', 'P(Xấp xỉ)': '{:.6f}', 'Sai số |P_exact - P_approx|': '{:.6f}'})
display(df_compare)

### Kịch bản: Tính **P(Letter | {'SAT': 'High', 'Difficulty': 'Easy', 'Intelligence': 'Low'})**

#### Suy luận Chính xác (Enumeration-Ask)

Điểm,Xác suất
Weak,0.487000
Strong,0.513000


Thời gian tính toán Chính xác: 0.001001 giây


#### Suy luận Xấp xỉ (Rejection Sampling)

**Số mẫu tạo ra:** 1000000 | **Số mẫu được chấp nhận:** 21004 (2.10%) 

Điểm,Xác suất
Weak,0.490811
Strong,0.509189


Thời gian tính toán Xấp xỉ: 8.082413 giây


### So sánh

Điểm,P(Chính xác),P(Xấp xỉ),Sai số |P_exact - P_approx|
Weak,0.487000,0.490811,0.003811
Strong,0.513000,0.509189,0.003811
